## IMPORT LIBRARY

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

## READING DATASET

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


RowNumber, CustomerId, Surname dont have any import role in analysis so we can ignore this

## INDEPENDENT AND DEPEND VARIABLE

In [3]:
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

In [4]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


### Create dummy variables

In [5]:
geography = pd.get_dummies(X['Geography'], drop_first=True)
gender = pd.get_dummies(X['Gender'], drop_first=True)

In [6]:
X = pd.concat([X, geography, gender], axis=1)
X = X.drop(['Geography', 'Gender'], axis=1)

In [7]:
X

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1


## TRAIN TEST SPLIT

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## STANDARD SCALER

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

## KERAS LIBRARY FOR ANN SETUP

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [11]:
# Initialising the ANN
classifier = Sequential()

In [12]:
# Adding the first hidden layer
classifier.add(Dense(6,  kernel_initializer = 'he_uniform', activation = 'relu', input_dim = 11))

In [13]:
# Adding the second hidden layer
classifier.add(Dense(6, kernel_initializer = 'he_uniform',activation='relu'))

In [14]:
# Adding the output layer
classifier.add(Dense(1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid'))

In [15]:
# Compiling the ANN
classifier.compile(optimizer = 'Adamax', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [16]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Fitting the ANN to the Training set
model_history=classifier.fit(X_train, y_train,validation_split=0.33, batch_size = 10, epochs = 100)

Epoch 1/100
536/536 [==============================] - 1s 1ms/step - loss: 0.6331 - accuracy: 0.7149 - val_loss: 0.5681 - val_accuracy: 0.7705
Epoch 2/100
536/536 [==============================] - 1s 968us/step - loss: 0.5538 - accuracy: 0.7750 - val_loss: 0.5268 - val_accuracy: 0.7823
Epoch 3/100
536/536 [==============================] - 0s 739us/step - loss: 0.5184 - accuracy: 0.7843 - val_loss: 0.5010 - val_accuracy: 0.7887
Epoch 4/100
536/536 [==============================] - 0s 685us/step - loss: 0.4947 - accuracy: 0.7890 - val_loss: 0.4819 - val_accuracy: 0.7910
Epoch 5/100
536/536 [==============================] - 0s 690us/step - loss: 0.4783 - accuracy: 0.7919 - val_loss: 0.4690 - val_accuracy: 0.7929
Epoch 6/100
536/536 [==============================] - 0s 694us/step - loss: 0.4664 - accuracy: 0.7955 - val_loss: 0.4595 - val_accuracy: 0.7974
Epoch 7/100
536/536 [==============================] - 0s 679us/step - loss: 0.4570 - accuracy: 0.7996 - val_loss: 0.4519 - val_accu

In [18]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred

array([[0.2550273 ],
       [0.24735153],
       [0.17066845],
       ...,
       [0.11201751],
       [0.09608349],
       [0.19294941]], dtype=float32)

In [19]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [20]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm 

array([[1531,   64],
       [ 216,  189]], dtype=int64)

In [21]:
# Calculate the Accuracy
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)
score

0.86